In [38]:
import pandas as pd
import numpy as np

In [39]:
# 예측값 불러오기
df1 = pd.read_csv("./testY/앞단 결과.csv", index_col=0)
df2 = pd.read_csv("./testY/앞단 결과.csv", index_col=0)

print(df1.shape, df2.shape)

(180, 930) (180, 930)


In [40]:
# 2800개 되게 열 드롭
df1.drop(columns="Date", inplace=True)
df1.shape

df2.drop(columns="Date", inplace=True)
df2.shape

(180, 929)

In [41]:
# 실제값 불러오기
df = pd.read_csv('./data/kospi_raw.csv', index_col=0)
# df.drop(columns='Date', inplace = True)
df.fillna(method='bfill', inplace=True)

# 마지막 3행을 추출합니다.
last_three_rows = df.tail(3)

# 모든 값이 같은지 확인하여 해당 열을 삭제합니다.
cols_to_drop = [col for col in df.columns if last_three_rows[col].nunique() == 1]

# 해당 열을 삭제합니다.
df.drop(columns=cols_to_drop, inplace=True)

df = df.tail(180)
df.shape

/var/folders/f9/6q54pn6j5cb3sqhqbmxdvs1c0000gn/T/ipykernel_19245/3385667629.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


(180, 929)

In [42]:
print(df1.shape)
print(df2.shape)
print(df.shape)

(180, 929)
(180, 929)
(180, 929)


In [43]:
# 열이름, 인덱스 같게
# df1.columns = df.columns
# df2.columns = df.columns

df1.index = df.index
df2.index = df.index

In [44]:
print(df1.isna().sum().sum())
print(df2.isna().sum().sum())
print(df.isna().sum().sum())

0
0
0


# 잔차 기반 점수(예측 정확도)

In [45]:
# Mean Absolute Error (MAE) 계산 함수
def calculate_mae(df_base, df_compare):
    df_base_np = df_base.values
    df_compare_np = df_compare.values
    mae = np.mean(np.abs(df_base_np - df_compare_np))
    return mae

# Mean Squared Error (MSE) 계산 함수
def calculate_mse(df_base, df_compare):
    df_base_np = df_base.values
    df_compare_np = df_compare.values
    mse = np.mean((df_base_np - df_compare_np) ** 2)
    return mse

In [46]:
# 오차 계산
mae_df1 = calculate_mae(df, df1)
mae_df2 = calculate_mae(df, df2)

mse_df1 = calculate_mse(df, df1)
mse_df2 = calculate_mse(df, df2)

In [47]:
print(f"df1에 대한 MAE: {mae_df1}\ndf1에 대한 MSE: {mse_df1}\n")
print(f"df2에 대한 MAE: {mae_df2}\ndf2에 대한 MSE: {mse_df2}")

df1에 대한 MAE: 37646.18037913995
df1에 대한 MSE: 7032081191.288865

df2에 대한 MAE: 37646.18037913995
df2에 대한 MSE: 7032081191.288865


# 순위 기반 점수

In [48]:
def calculate_rank_score(df_base, df_compare):
    
    # df_base 전처리
    new_columns1 = [f'col_{i//5}' for i in range(0, df_base.shape[1], 5)]  # 새로운 열 이름을 생성합니다.
    df_score_base = pd.DataFrame()

    # 5의 배수인지 화긴~
    num_rows = df_base.shape[0]
    if num_rows % 5 != 0:
        print("Warning: The number of rows is not a multiple of 5. The last few rows will be dropped.(df_base)")

    num_rows_adjusted = (num_rows // 5) * 5
    df_base_adjusted = df_base.iloc[:num_rows_adjusted]

    # 변화율 계산
    df_diff_base = (df_base_adjusted.groupby(np.arange(num_rows_adjusted) // 5).last() - df_base_adjusted.groupby(np.arange(num_rows_adjusted) // 5).first()) / df_base_adjusted.groupby(np.arange(num_rows_adjusted) // 5).first()

    df_diff_base.index = df_base_adjusted.index[np.arange(num_rows_adjusted) % 5 == 0]

    # 미분값 바탕으로 순위 구하기
    df_base_ranked = df_diff_base.rank(axis=1, method='average')
    df_base_ranked = df_base_ranked.transpose()
    
    
    # df_compare 전처리
    new_columns2 = [f'col_{i//5}' for i in range(0, df_compare.shape[1], 5)]  # 새로운 열 이름을 생성합니다.
    df_score_compare = pd.DataFrame()

    # 5의 배수인지 화긴~
    num_rows = df_compare.shape[0]
    if num_rows % 5 != 0:
        print("Warning: The number of rows is not a multiple of 5. The last few rows will be dropped.(df_base)")

    num_rows_adjusted = (num_rows // 5) * 5
    df_compare_adjusted = df_compare.iloc[:num_rows_adjusted]

    # 변화율 계산
    df_diff_compare = (df_compare_adjusted.groupby(np.arange(num_rows_adjusted) // 5).last() - df_compare_adjusted.groupby(np.arange(num_rows_adjusted) // 5).first()) / df_compare_adjusted.groupby(np.arange(num_rows_adjusted) // 5).first()

    df_diff_compare.index = df_compare_adjusted.index[np.arange(num_rows_adjusted) % 5 == 0]

    # 미분값 바탕으로 순위 구하기
    df_compare_ranked = df_diff_compare.rank(axis=1, method='average')
    df_compare_ranked = df_compare_ranked.transpose()

    # 순위
    df_top6 = df_compare_ranked.apply(lambda x: x.nsmallest(6).index)

    # 결과 출력
    df_top6

    # 새로운 데이터프레임을 초기화합니다.
    df_result = pd.DataFrame()

    # df1의 각 열에 대해 반복합니다.
    for col in df_top6.columns:
    # df1에서 현재 열의 인덱스를 가져옵니다.
        indices = df_top6[col]
        # df2에서 해당 인덱스의 값을 제곱하여 더합니다.
        df_result[col] = [df_base_ranked.loc[indices, col].pow(2).sum()]
    
    # 숫자 너무 크니 루트 씌우기    
    df_result = df_result.apply(np.sqrt)

    # 모든 값을 평균내어 새로운 열을 추가합니다.
    df_result['평균'] = df_result.mean(axis=1)
    return df_result, df_top6

In [49]:
df1_rank_score, df1_top6 = calculate_rank_score(df, df1)
df2_rank_score, df2_top6 = calculate_rank_score(df, df2)

In [50]:
df1_rank_score

,2023-08-28,2023-09-04,2023-09-11,2023-09-18,2023-09-25,2023-10-06,2023-10-16,2023-10-23,2023-10-30,2023-11-06,...,2024-03-19,2024-03-26,2024-04-02,2024-04-09,2024-04-17,2024-04-24,2024-05-02,2024-05-10,2024-05-20,평균
0,1261.596607,1362.324851,1082.265217,1368.050438,982.29998,997.385582,1235.181161,1581.533749,631.001585,1540.303542,...,1974.256062,686.174176,1028.110403,1083.617552,1117.524049,1359.423039,1138.382625,1134.222641,1125.308402,1241.736986


In [51]:
df2_rank_score

,2023-08-28,2023-09-04,2023-09-11,2023-09-18,2023-09-25,2023-10-06,2023-10-16,2023-10-23,2023-10-30,2023-11-06,...,2024-03-19,2024-03-26,2024-04-02,2024-04-09,2024-04-17,2024-04-24,2024-05-02,2024-05-10,2024-05-20,평균
0,1261.596607,1362.324851,1082.265217,1368.050438,982.29998,997.385582,1235.181161,1581.533749,631.001585,1540.303542,...,1974.256062,686.174176,1028.110403,1083.617552,1117.524049,1359.423039,1138.382625,1134.222641,1125.308402,1241.736986


In [52]:
df1_top6

Date,2023-08-28,2023-09-04,2023-09-11,2023-09-18,2023-09-25,2023-10-06,2023-10-16,2023-10-23,2023-10-30,2023-11-06,...,2024-03-12,2024-03-19,2024-03-26,2024-04-02,2024-04-09,2024-04-17,2024-04-24,2024-05-02,2024-05-10,2024-05-20
0,012170,012170,012170,012170,012170,012170,093240,012170,012170,118000,...,012170,118000,019175,168490,36328K,012170,012170,093240,006345,012170
1,093240,118000,118000,033250,118000,118000,003520,093240,002720,45226K,...,003520,007460,012170,012170,093240,093240,093240,012170,012170,093240
2,118000,005360,093240,118000,093240,093240,118000,118000,45226K,012170,...,118000,093240,093240,093240,45226K,45226K,45226K,45226K,093240,45226K
3,003520,093240,003520,003520,003520,45226K,45226K,45226K,093240,093240,...,093240,003520,002720,118000,003520,003520,003520,003520,45226K,003520
4,285130,003520,285130,093240,45226K,003520,012170,003520,003520,003520,...,45226K,45226K,010660,003520,000040,007460,003620,118000,003520,006345
5,021050,004105,001070,285130,285130,003680,007460,033250,118000,007460,...,001070,012170,003520,45226K,118000,118000,006345,119650,119650,168490
